<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/Matching_entities_Bert_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download dependencies

In [17]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.5MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 46.8MB/s 
     |████████████████████████████████| 901kB 49.5MB/s 


## load datasets 

In [27]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [12]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [13]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [33]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [88]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    train_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    train_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 
    test_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 
    valid_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= zip(train_data['text_left'],train_data['text_right'])
    test_data['tuples']= zip(test_data['text_left'],test_data['text_right'])
    valid_data['tuples']= zip(valid_data['text_left'],valid_data['text_right'])

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [89]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

,label,text_left,text_right,tuples
0,0,microsoft visio standard 2007 version upgrade....,adobe cs3 design standard upgrade. [attribute]...,<zip object at 0x7f0dd19f6870>
1,0,adobe premiere pro cs3 upgrade. [attribute] ad...,adobe cs3 design standard upgrade. [attribute]...,<zip object at 0x7f0dd19f6870>
2,0,adobe indesign cs3 upgrade from pagemaker [ ma...,adobe cs3 design standard upgrade. [attribute]...,<zip object at 0x7f0dd19f6870>
3,0,adobe creative suite cs3 design standard [ mac...,adobe cs3 design standard upgrade. [attribute]...,<zip object at 0x7f0dd19f6870>
4,0,adobe creative suite cs3 design premium upsell...,adobe cs3 design standard upgrade. [attribute]...,<zip object at 0x7f0dd19f6870>


In [91]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [92]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':'merged_data/amazon_google/train.csv',
                                              'test':'merged_data/amazon_google/test.csv',
                                              'valid':'merged_data/amazon_google/valid.csv'})

Using custom data configuration default-be0bf4698c1f6c57


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-be0bf4698c1f6c57/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [77]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [93]:
def tokenize_function(examples):
    # examples['text_left'].head()
    # examples['tuples'] = examples[["text_left","text_right"]].apply(lambda x: (x[0],x[1]),axis=1)
    return tokenizer(examples['tuples'], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [101]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 6874
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 2293
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 2293
    })
})

In [105]:
import torch
torch.cuda.empty_cache()

In [106]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfo

In [107]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [110]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [111]:
trainer.train()

***** Running training *****
  Num examples = 6874
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2580


Step,Training Loss
500,0.335400
1000,0.345300
1500,0.333100
2000,0.339100
2500,0.329400


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your mod

TrainOutput(global_step=2580, training_loss=0.33767145585644154, metrics={'train_runtime': 2068.1287, 'train_samples_per_second': 9.971, 'train_steps_per_second': 1.248, 'total_flos': 6861638079959040.0, 'train_loss': 0.33767145585644154, 'epoch': 3.0})

In [117]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [118]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2293
  Batch size = 8


{'eval_accuracy': 0.8979502834714348,
 'eval_loss': 0.3302627503871918,
 'eval_runtime': 81.9158,
 'eval_samples_per_second': 27.992,
 'eval_steps_per_second': 3.504}